import所需要的元件
request:用來取得html檔案
time:延遲時間
os:建立資料夾
BeautifulSoup:可解析html檔案 利用標籤選擇器篩選資料
以下網址有關於BeautifulSoup的教學參考
https://www.youtube.com/watch?v=yG8-mKyLmNw

In [ ]:
import requests
import time
import os
from bs4 import BeautifulSoup

由於原網址清空BOX了暫時抓不到

In [ ]:
listurl='https://www.knbsb.nl/competitie/competities/honkbal-competities/!/uitslagen/H100/honkbal-hoofdklasse/reguliere-competitie'
res = requests.get(listurl)
soup1 = BeautifulSoup(res.text,'lxml')
ops = soup1.find_all('option')
for op in ops:
    listurl='https://www.knbsb.nl/competitie/competities/honkbal-competities/!/uitslagen/'+op['value']
    folder = op['value'].split('/')[2]
    print(folder)
    try:
        os.mkdir(folder)
    except IOError:
        print("Error: folder already exixt")
    res = requests.get(listurl);
    soup2 = BeautifulSoup(res.text,'lxml')
    boxes=soup2.find_all(attrs={'class':'icon'})
    for box in boxes:
        url=box.a['href']
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'lxml')
        print(soup.title.string)
        print(soup.find_all(attrs={'size':'3'})[1].text.split('\n')[2]+'\n')
        logs=soup.find_all(attrs={'cellpadding':'3'})
        
        try:
            fo = open(os.getcwd()+"\\"+folder+"\\"+soup.title.string+".txt", "w")
        except IOError:
            print("Error: cannot open file")

        for log in logs:
            print(log.b.text.replace(" -",''))
            print(log.text.replace(log.b.text,'').replace(log.i.text,'').strip().replace('\n',' ').replace(". ",".\n").replace("; ",";\n"))
            print(log.i.text.replace('\n',' ')+'\n')
            fo.write(log.b.text.replace(" -",'')+'\n');
            fo.write(log.text.replace(log.b.text,'').replace(log.i.text,'').strip().replace('\n',' ').replace(". ",".\n").replace("; ",";\n")+'\n')
            fo.write(log.i.text.replace('\n',' ')+'\n')
        fo.write(soup.find_all(attrs={'size':'3'})[1].text.split('\n')[2]+'\n')
        fo.close()
        time.sleep(0.1)



由庫存網頁抓取過去的比賽資料網址

find_all:將所有符合條件的資料依照順序做成一個List檔案
attrs={'class':'icon'} :篩選條件attribute為class='icon'的網址資料
再由迴圈將url印出確認

In [ ]:
listurl='https://webcache.googleusercontent.com/search?q=cache:Mjmq83yWFgYJ:https://www.knbsb.nl/competitie/competities/honkbal-competities/!/uitslagen/H100/honkbal-hoofdklasse/reguliere-competitie+&cd=1&hl=zh-TW&ct=clnk&gl=tw'
res = requests.get(listurl);
soup = BeautifulSoup(res.text,'lxml')
boxes=soup.find_all(attrs={'class':'icon'})
for box in boxes:
        url=box.a['href']
        print(url)

由上面抓到的網址List來將每個比賽的play by play抓下來



In [ ]:
for box in boxes:
    url=box.a['href']
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')

    print(soup.title.string))             #比賽標題包含隊伍和日期
    print(soup.find_all(attrs={'size':'3'})[1].text.split('\n')[2]+'\n')  #雙方比分
    logs=soup.find_all(attrs={'cellpadding':'3'})   #playbyplay list

    try:
       os.mkdir("reguliere-competitie")          #建立該賽季資料夾
    except IOError:
        print("Error: folder already exixt") 

    fo = open(os.getcwd()+"\\reguliere-competitie\\"+soup.title.string+".txt", "w") #以標題作為檔名
    
    for log in logs:
        print(log.b.text.replace(" -",''))   #b標籤為某隊伍第幾局 去掉後面多餘的部分
        #底下為playlog 將b標籤和i標籤的資料用replace去除掉 strip用來去掉換行空格 接著再用replace整理句子
        print(log.text.replace(log.b.text,'').replace(log.i.text,'').strip().replace('\n',' ').replace(". ",".\n").replace("; ",";\n"))
        print(log.i.text.replace('\n',' ')+'\n')#i標籤為該局的結果
        #底下將資料寫入檔案
        fo.write(log.b.text.replace(" -",'')+'\n');
        fo.write(log.text.replace(log.b.text,'').replace(log.i.text,'').strip().replace('\n',' ').replace(". ",".\n").replace("; ",";\n")+'\n')
        fo.write(log.i.text.replace('\n',' ')+'\n')
    fo.write(soup.find_all(attrs={'size':'3'})[1].text.split('\n')[2]+'\n')  #於最後寫入隊伍比分
    fo.close()    #關檔 換下一場比賽
